<a href="https://colab.research.google.com/github/notomasz/MZ/blob/master/covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Normalizacja rozmiaru zdjęć**

In [ ]:
import PIL
import os
import os.path
from PIL import Image

f = r'/content/drive/My Drive/data/'
fs = r'/content/drive/My Drive/data/'
for file in os.listdir(f):
    f_img = f+"/"+file
    img = Image.open(f_img)
    img = img.resize((224,224))
    img.save(fs+'/'+file, "PNG")

### **Wczytanie bibliotek**

In [ ]:
#instalacja
!pip install tf-nightly

#google drive
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

#numpy, pandas, matplotlib, seaborn
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

#tensorflow/keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Input, Dense, Dropout, Conv2D, MaxPooling2D, Activation, BatchNormalization, Flatten, add, GlobalAveragePooling2D)
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import binary_accuracy
from tensorflow.keras.optimizers import Adam, RMSprop
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import Xception
from keras.applications import ResNet50V2
from tensorflow.keras.models import load_model

#sklearn
from sklearn.metrics import f1_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve, auc


### **Zdefiniowanie funkcji oraz zmiennych globalnych**

In [ ]:
def print_layers(model):
    for idx, layer in enumerate(model.layers):
        print("layer {}: {}, trainable: {}".format(idx, layer.name, layer.trainable))

def summary(y_test, y_pred):
  auc = roc_auc_score(y_test, y_pred)
  ax2 = plt.figure(figsize=(12,8))
  fpr, tpr, thresholds = roc_curve(y_test, y_pred)
  plt.plot(fpr, tpr, label='Model (area = %0.2f)' % auc, c='#1f77b4')
  plt.plot([0, 1], [0, 1],'r--')
  plt.xlabel('1 - specyficzność')
  plt.ylabel('Czułość')
  #plt.title('Receiver operating characteristic')
  plt.legend(loc="lower right")
  ax = plt.figure(figsize=(12,8))
  cm = confusion_matrix(y_test, y_pred)
  sns.set(font_scale=1.3)
  ax = sns.heatmap(cm, vmin=0, vmax=500,annot=True,linewidths=.1, fmt='d', cmap='Blues', xticklabels=['nie-covid', 'covid'], yticklabels=['nie-covid', 'covid'])
  plt.xlabel("Klasa predykowana",)
  plt.ylabel("Klasa właściwa")
  tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
  accuracy= (tp+tn)/ (tp+tn+fp+fn) * 100
  specificity = tn/(tn+fp) * 100
  sensitivity = tp/(tp+fn) * 100
  precision = tp/(tp+fp) * 100
  f1score = f1_score(y_test, y_pred) * 100
  print('Accuracy: {}\nSensitivity: {}\nSpecificity: {}\nAUC: {}\nF1_score: {}\nPrecision: {}'.format(accuracy, sensitivity, specificity, auc, f1score, precision))
  return ax, ax2

def create_model(model_name, lr, dropout):
  model = Sequential()
  model.add(model_name)
  model.add(Dense(512, activation='relu', input_dim=(2663,100352), name='Warstwa_gesta_1'))
  model.add(Dropout(dropout, name='Porzucenie_1'))
  model.add(Dense(512, activation='relu', name='Warstwa_gesta_2'))
  model.add(Dropout(dropout, name='Porzucenie_2'))
  model.add(Dense(256, activation='relu', name='Warstwa_gesta_3'))
  model.add(Dense(128, activation='relu', name='Warstwa_gesta_4'))
  model.add(Dense(1, activation='sigmoid', name='Warstwa_wyjsciowa'))

  model.compile(loss='binary_crossentropy',
                optimizer=Adam(lr=lr),
                metrics=['binary_accuracy'])
  return model

#rozmiar danych wejsciowych
input_shape=(224,224,3)

#wagi klas
class_weight = {0:1, 1:3.01659125189}

#zapisywanie modeli oraz przerywanie uczenia
callbacks_list_1 = [EarlyStopping(monitor='val_loss', patience=10),
        ModelCheckpoint(filepath='/content/drive/My Drive/saved_models/Xception_1.h5', monitor='val_loss', save_best_only=True, mode='min')]

callbacks_list_2 = [EarlyStopping(monitor='val_loss', patience=10),
        ModelCheckpoint(filepath='/content/drive/My Drive/saved_models/Xception_2.h5', monitor='val_loss', save_best_only=True, mode='min')]

callbacks_list_3 = [EarlyStopping(monitor='val_loss', patience=10),
        ModelCheckpoint(filepath='/content/drive/My Drive/saved_models/ResNet_1.h5', monitor='val_loss', save_best_only=True, mode='min')]

callbacks_list_4 = [EarlyStopping(monitor='val_loss', patience=10),
        ModelCheckpoint(filepath='/content/drive/My Drive/saved_models/ResNet_2.h5', monitor='val_loss', save_best_only=True, mode='min')]

### **Zdefiniowanie parametrów augmentacji**

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.1, rotation_range=10,
                                   width_shift_range=0.10, shear_range=0.1, 
                                   horizontal_flip=True, fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/final_resized/train/',
    class_mode="binary",
    #class_names=['covid', 'non'],
    color_mode="rgb",
    batch_size=64,
    target_size=(224, 224),
    shuffle=True,
    seed=111,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
)
val_generator = val_datagen.flow_from_directory(
    '/content/drive/My Drive/final_resized/validation/',
    class_mode="binary",
    #class_names=['covid', 'non'],
    color_mode="rgb",
    batch_size=64,
    target_size=(224, 224),
    shuffle=True,
    seed=111,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
)

test_generator = val_datagen.flow_from_directory(
    '/content/drive/My Drive/final_resized/test/',
    class_mode="binary",
    #class_names=['covid', 'non'],
    color_mode="rgb",
    batch_size=64,
    target_size=(224, 224),
    shuffle=False,
    seed=111,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
)

Found 2663 images belonging to 2 classes.
Found 50 images belonging to 2 classes.
Found 2800 images belonging to 2 classes.


### **Xception**

#### ostatni moduł

In [ ]:
xception = Xception(include_top=False, weights='imagenet', 
                                     input_shape=input_shape, pooling='max')

output = xception.layers[-1].output
output = Flatten()(output)
xception_model = Model(xception.input, output, name='Baza_konwolucyjna')

xception_model.trainable = True
for layer in xception_model.layers:
    layer.trainable = False
    
#bottleneck_feature_example = res.predict(x_train.take(21))
#print(bottleneck_feature_example.shape)
#plt.imshow(bottleneck_feature_example[0][:,:,0])

for layer in xception_model.layers[-8:]:
  layer.trainable=True
print_layers(xception_model)

In [ ]:
xception_model.summary()

Model: "Baza_konwolucyjna"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________

#### dwa ostatnie moduły

In [ ]:
xception2 = Xception(include_top=False, weights='imagenet', 
                                     input_shape=input_shape)

output = xception2.layers[-1].output
output = Flatten()(output)
xception_model2= Model(xception2.input, output, name='Baza_konwolucyjna')

xception_model2.trainable = True
for layer in xception_model2.layers:
    layer.trainable = False
    
#bottleneck_feature_example = res.predict(x_train.take(21))
#print(bottleneck_feature_example.shape)
#plt.imshow(bottleneck_feature_example[0][:,:,0])

for layer in xception_model2.layers[-27:]:
  layer.trainable=True
print_layers(xception_model2)

### **ResNet**

##### ostatni moduł

In [ ]:
res = ResNet50V2(include_top=False, weights='imagenet', 
                                     input_shape=input_shape)

output = res.layers[-1].output
#output = GlobalAveragePooling2D()(output)
output = Flatten()(output)
res_model = Model(res.input, output ,name='Baza_konwolucyjna')

res_model.trainable = True
for layer in res_model.layers:
    layer.trainable = False
    

#bottleneck_feature_example = res.predict(x_train.take(5))
#print(bottleneck_feature_example.shape)
#plt.imshow(bottleneck_feature_example[0][:,:,0])

for layer in res_model.layers[-14:]:
  layer.trainable=True
print_layers(res_model)

#### dwa ostatnie moduły

In [ ]:
res2 = ResNet50V2(include_top=False, weights='imagenet', 
                                     input_shape=input_shape)

output = res2.layers[-1].output
#output = GlobalAveragePooling2D()(output)
output = Flatten()(output)
res_model2 = Model(res2.input, output ,name='Baza_konwolucyjna')

res_model2.trainable = True
for layer in res_model2.layers:
    layer.trainable = False
    
#bottleneck_feature_example = res.predict(x_train.take(1))
#print(bottleneck_feature_example.shape)
#plt.imshow(bottleneck_feature_example[0][:,:,0])

for layer in res_model2.layers[-25:]:
  layer.trainable=True
print_layers(res_model2)

layer 0: input_2, trainable: False
layer 1: conv1_pad, trainable: False
layer 2: conv1_conv, trainable: False
layer 3: pool1_pad, trainable: False
layer 4: pool1_pool, trainable: False
layer 5: conv2_block1_preact_bn, trainable: False
layer 6: conv2_block1_preact_relu, trainable: False
layer 7: conv2_block1_1_conv, trainable: False
layer 8: conv2_block1_1_bn, trainable: False
layer 9: conv2_block1_1_relu, trainable: False
layer 10: conv2_block1_2_pad, trainable: False
layer 11: conv2_block1_2_conv, trainable: False
layer 12: conv2_block1_2_bn, trainable: False
layer 13: conv2_block1_2_relu, trainable: False
layer 14: conv2_block1_0_conv, trainable: False
layer 15: conv2_block1_3_conv, trainable: False
layer 16: conv2_block1_out, trainable: False
layer 17: conv2_block2_preact_bn, trainable: False
layer 18: conv2_block2_preact_relu, trainable: False
layer 19: conv2_block2_1_conv, trainable: False
layer 20: conv2_block2_1_bn, trainable: False
layer 21: conv2_block2_1_relu, trainable: Fals

### **Finalne modele**

In [ ]:
#Xception ostatni blok
xception_final_1 = create_model(xception_model, lr=1e-5, dropout=0.3)

#Xception dwa ostatnie bloki
xception_final_2 = create_model(xception_model2, lr=1e-5, dropout=0.3)

#ResNet odmrożony ostatni blok
resnet_final_1 = create_model(res_model, lr=1e5, dropout=0.3)

#ResNet dwa ostatnie bloki
resnet_final_2 = create_model(res_model2, lr=1e5, dropout=0.3)

In [ ]:
xception_final_1.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Baza_konwolucyjna (Functiona (None, 2048)              20861480  
_________________________________________________________________
Warstwa_gesta_1 (Dense)      (None, 512)               1049088   
_________________________________________________________________
Porzucenie_1 (Dropout)       (None, 512)               0         
_________________________________________________________________
Warstwa_gesta_2 (Dense)      (None, 512)               262656    
_________________________________________________________________
Porzucenie_2 (Dropout)       (None, 512)               0         
_________________________________________________________________
Warstwa_gesta_3 (Dense)      (None, 256)               131328    
_________________________________________________________________
Warstwa_gesta_4 (Dense)      (None, 128)              

### **Trening**

In [ ]:
xception_1_history = xception_final_1.fit(train_generator, batch_size=64, validation_data=val_generator, 
                    steps_per_epoch=42, validation_steps=1, validation_batch_size=50, epochs=50,
                    callbacks=callbacks_list_1, class_weight=class_weight)

xception_2_history = xception_final_2.fit(train_generator, batch_size=64, validation_data=val_generator, 
                    steps_per_epoch=42, validation_steps=1, validation_batch_size=50, epochs=50,
                    callbacks=callbacks_list_2, class_weight=class_weight)

resnet_1_history = resnet_final_1.fit(train_generator, batch_size=64, validation_data=val_generator, 
                    steps_per_epoch=42, validation_steps=1, validation_batch_size=50, epochs=50,
                    callbacks=callbacks_list_3, class_weight=class_weight)

resnet_2_history = resnet_final_2.fit(train_generator, batch_size=64, validation_data=val_generator, 
                    steps_per_epoch=42, validation_steps=1, validation_batch_size=50, epochs=50,
                    callbacks=callbacks_list_4, class_weight=class_weight)

Epoch 1/50
42/42 [==============================] - 681s 16s/step - loss: 1.1271 - binary_accuracy: 0.6754 - val_loss: 0.6558 - val_binary_accuracy: 0.6400
Epoch 2/50
42/42 [==============================] - 705s 17s/step - loss: 0.9981 - binary_accuracy: 0.6248 - val_loss: 0.6051 - val_binary_accuracy: 0.8000
Epoch 3/50
42/42 [==============================] - 708s 17s/step - loss: 0.9416 - binary_accuracy: 0.6955 - val_loss: 0.5549 - val_binary_accuracy: 0.8000
Epoch 4/50
42/42 [==============================] - 699s 17s/step - loss: 0.8982 - binary_accuracy: 0.6974 - val_loss: 0.5186 - val_binary_accuracy: 0.7400
Epoch 5/50
42/42 [==============================] - 701s 17s/step - loss: 0.8447 - binary_accuracy: 0.7601 - val_loss: 0.4666 - val_binary_accuracy: 0.8000
Epoch 6/50
42/42 [==============================] - 712s 17s/step - loss: 0.7715 - binary_accuracy: 0.7832 - val_loss: 0.4007 - val_binary_accuracy: 0.8200
Epoch 7/50
42/42 [==============================] - 708s 17s/ste

### **Testowanie oraz ewaluacja wyników**

In [ ]:
y_test = [e for e in test_generator.labels]

In [ ]:
path_xception_1 = '/content/drive/My Drive/mgr/xception_1/xception_1.h5'
path_xception_2 = '/content/drive/My Drive/mgr/xception_2/xception_2.h5'

path_resnet_1 = '/content/drive/My Drive/mgr/ResNet_1/resnet_1.h5'
path_resnet_2 = '/content/drive/My Drive/mgr/resnet_2/resnet_2.h5'


xception_1 = load_model(path_xception_1)
xception_2 = load_model(path_xception_2)
resnet_1 = load_model(path_resnet_1)
resnet_2 = load_model(path_resnet_2)

In [ ]:
y_pred_xception_1 = (xception_1.predict(test_generator) >= 0.5).astype("int32")
y_pred_xception_2 = (xception_2.predict(test_generator) >= 0.5).astype("int32")

y_pred_resnet_1 = (resnet_1.predict(test_generator) >= 0.5).astype("int32")
y_pred_resnet_2 = (resnet_2.predict(test_generator) >= 0.5).astype("int32")

KeyboardInterrupt: ignored

In [ ]:
summary(y_test, y_pred_xception_1)
summary(y_test, y_pred_xception_2)

summary(y_test, y_pred_resnet_1)
summary(y_test, y_pred_resnet_2)

In [ ]:
training = pd.read_csv('/content/drive/My Drive/results.csv', sep=';')
training = training[:].astype('float32')

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(12,8))
#plt.title('Strata - Resnet50',fontsize=16, fontweight='bold')
plt.xlabel('Epoka')
plt.ylabel('Dokładność', rotation=90)
plt.plot(training['x1_Acc'], lw=3, label='Xception_1')
plt.plot(training['x2_Acc'], lw=3, label='Xception_2')
plt.plot(training['r1_Acc'], lw=3, label='ResNet_1')
plt.plot(training['r2_Acc'], lw=3, label='ResNet_2')
plt.plot()
plt.legend(loc=4, prop = {'size':14})

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(12,8))
#plt.title('Strata - Resnet50',fontsize=16, fontweight='bold')
plt.xlabel('Epoka')
plt.ylabel('Strata', rotation=90)
plt.plot(training['x_1_Loss'], lw=3, label='Xception_1')
plt.plot(training['x2_Loss'], lw=3, label='Xception_2')
plt.plot(training['r1_Loss'], lw=3, label='ResNet_1')
plt.plot(training['r2_Loss'], lw=3, label='ResNet_2')
plt.legend(loc=1, prop = {'size':14})

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(12,8))
#plt.title('Strata - Resnet50',fontsize=16, fontweight='bold')
plt.xlabel('Epoka')
plt.ylabel('Dokładność', rotation=90)
plt.plot(training['x1_Val_acc'], lw=3, label='Xception_1')
plt.plot(training['x2_Val_acc'], lw=3, label='Xception_2')
plt.plot(training['r1_Val_acc'], lw=3, label='ResNet_1')
plt.plot(training['r2_Val_acc'], lw=3, label='ResNet_2')
plt.plot()
plt.legend(loc=4, prop = {'size':14})

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(12,8))
#plt.title('Strata - Resnet50',fontsize=16, fontweight='bold')
plt.xlabel('Epoka')
plt.ylabel('Strata', rotation=90)
plt.plot(training['x1_Val_loss'], lw=3, label='Xception_1')
plt.plot(training['x2_Val_loss'], lw=3, label='Xception_2')
plt.plot(training['r1_Val_loss'], lw=3, label='ResNet_1')
plt.plot(training['r2_Val_loss'], lw=3, label='ResNet_2')
plt.ylim(0,10)
plt.legend(loc=4, prop = {'size':14})

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(12,8))
auc1 = roc_auc_score(y_test, y_pred_xception_1)
auc2 = roc_auc_score(y_test, y_pred_xception_2)
auc3 = roc_auc_score(y_test, y_pred_resnet_1)
auc4 = roc_auc_score(y_test, y_pred_resnet_2)
plt.figure(figsize=(12,8))
fpr1, tpr1, thresholds = roc_curve(y_test, y_pred_xception_1)
fpr2, tpr2, thresholds = roc_curve(y_test, y_pred_xception_2)
fpr3, tpr3, thresholds = roc_curve(y_test, y_pred_resnet_1)
fpr4, tpr4, thresholds = roc_curve(y_test, y_pred_resnet_2)

plt.plot(fpr1, tpr1, label='Xception_1 (AUC = %0.4f)' % auc1, lw=3)
plt.plot(fpr2, tpr2, label='Xception_2 (AUC = %0.4f)' % auc2, lw=3)
plt.plot(fpr3, tpr3, label='ResNet_1 (AUC = %0.4f)' % auc3, lw=3)
plt.plot(fpr4, tpr4, label='ResNet_2 (AUC = %0.4f)' % auc3, lw=3)
plt.legend()
plt.xlabel('1 - specyficzność')
plt.ylabel('Czułość');

In [ ]:
precision_x1, recall_x1, _ = precision_recall_curve(y_test, y_pred_xception_1)
auc1 = auc(recall_x1, precision_x1)

precision_x2, recall_x2, _ = precision_recall_curve(y_test, y_pred_xception_2)
auc2 = auc(recall_x2, precision_x2)

precision_r1, recall_r1, _ = precision_recall_curve(y_test, y_pred_resnet_1)
auc3 = auc(recall_r1, precision_r1)

precision_r2, recall_r2, _ = precision_recall_curve(y_test, y_pred_resnet_2)
auc4 = auc(recall_r2, precision_r2)

plt.figure(figsize=(12,8))
fpr1, tpr1, thresholds = precision_recall_curve(y_test, y_pred_xception_1)
fpr2, tpr2, thresholds = precision_recall_curve(y_test, y_pred_xception_2)
fpr3, tpr3, thresholds = precision_recall_curve(y_test, y_pred_resnet_1)
plt.
plt.plot(precision_x1, recall_x1, label='Xception_1 (AUC = %0.4f)' % auc1, lw=3)
plt.plot(precision_x2, recall_x2, label='Xception_2 (AUC = %0.4f)' % auc2, lw=3)
plt.plot(precision_r1, recall_r1, label='ResNet_1 (AUC = %0.4f)' % auc3, lw=3)
plt.plot(precision_r2, recall_r2, label='ResNet_2 (AUC = %0.4f)' % auc4, lw=3)
plt.legend()
plt.xlabel('Czułość')
plt.ylabel('Precyzja');